# Reference

In [64]:
import numpy as np

w = 1024
h = 768
canvas = np.zeros((h, w))

line_widths = [0 for i in range(h)]

actual = 0
count = 0

while True:
    i = np.argmin(line_widths)
    binary_count = [int(bit) for bit in reversed(bin(count)[2:])]
    offset = line_widths[i]
    if offset + len(binary_count) >= w:
        break
    canvas[i, offset:offset+len(binary_count)] = binary_count
    line_widths[i] += len(binary_count)
    count += 1
    if actual % 255 == 0:
        count >>= 1
    actual += 1



# Optimized

In [103]:
import numpy as np
from numba import njit

@njit
def int_to_binary_array(value):
    binary_array = []
    while value > 0:
        binary_array.append(value % 2)
        value //= 2
    return binary_array

@njit
def generate_canvas(w, h, skip):
    canvas = np.zeros((h, w), dtype=np.float32)
    line_widths = np.zeros(h, dtype=np.int32)
    actual = 0
    count = 0
    
    binary_arrays = [int_to_binary_array(i) for i in range(2048)]

    while True:
        i = np.argmin(line_widths)
        binary_count = binary_arrays[count] #int_to_binary_array(count)
        offset = line_widths[i]
        if offset + len(binary_count) >= w:
            break
        canvas[i, offset:offset + len(binary_count)] = binary_count
        line_widths[i] += len(binary_count)
        count += 1
        if actual % skip == 0:
            count >>= 1
        actual += 1

    return canvas

w = 1920*4
h = 1080*4
skip = 1000
canvas = generate_canvas(w, h, skip)

# Heap-based optimization

In [114]:
import numpy as np
import heapq
from utils.imutil import imwrite

memo = {}

def int_to_binary_array(value):
    if value in memo:
        return memo[value]
    
    binary_array = []
    original_value = value
    while value > 0:
        binary_array.append(bool(value % 2))
        value //= 2
    memo[original_value] = binary_array
    return binary_array

def generate_canvas_optimized(w, h, skip):
    canvas = np.zeros((h, w), dtype=bool)
    line_widths = np.zeros(h, dtype=np.int32)
    heap = [(0, i) for i in range(h)]
    heapq.heapify(heap)
    
    actual = 0
    count = 0

    while True:
        min_width, i = heapq.heappop(heap)
        binary_count = int_to_binary_array(count)
        offset = line_widths[i]
        if offset + len(binary_count) >= w:
            break
        canvas[i, offset:offset + len(binary_count)] = binary_count
        line_widths[i] += len(binary_count)
        heapq.heappush(heap, (line_widths[i], i))
        count += 1
        if actual % skip == 0:
            count >>= 1
        actual += 1

    return canvas

for i in range(768,1023):
    w = 1920 * 4
    h = 1080 * 4
    skip = i
    canvas = generate_canvas_optimized(w, h, skip)
    imwrite(f"{w}x{h}-{skip}.png", canvas*255)

KeyboardInterrupt: 

In [90]:
# from utils.imutil import imshow

# imshow(canvas[:768,:1024]*255)